<img src="https://www.th-koeln.de/img/logo.svg" style="float:right;" width="200">

# 13th exercise: <font color="#C70039">First Reinforcement Learning Q-Table learning</font>
* Course: AML
* Lecturer: <a href="https://www.gernotheisenberg.de/">Gernot Heisenberg</a>
* Author of notebook: <a href="https://www.gernotheisenberg.de/">Gernot Heisenberg</a>
    * Date:   01.04.2026
    * Student: Tim Voßmerbäumer
    * Matr.Nr.: 11474232

---------------------------------
**GENERAL NOTE 1**: 
Please make sure you are reading the entire notebook, since it contains a lot of information on your tasks (e.g. regarding the set of certain paramaters or a specific computational trick), and the written mark downs as well as comments contain a lot of information on how things work together as a whole. 

**GENERAL NOTE 2**: 
* Please, when commenting source code, just use English language only. 
* When describing an observation please use English language, too.
* This applies to all exercises throughout this course.

---------------------------------

### <font color="FFC300">TASKS</font>:
The tasks that you need to work on within this notebook are always indicated below as bullet points. 
If a task is more challenging and consists of several steps, this is indicated as well. 
Make sure you have worked down the task list and commented your doings. 
This should be done by using markdown.<br> 
<font color=red>Make sure you don't forget to specify your name and your matriculation number in the notebook.</font>

**YOUR TASKS in this exercise are as follows**:
1. import the notebook to Google Colab or use your local machine.
2. make sure you specified you name and your matriculation number in the header below my name and date. 
    * set the date too and remove mine.
3. read the entire notebook carefully 
    * add comments whereever you feel it necessary for better understanding
    * run the notebook for the first time. 
4. play with all hyperparameters including the actions, states, rewards table.
5. add and implement an ϵ-greedy strategy 
---------------------------------

In [1]:
# Imports 
import numpy as np

### Create the possible states

In [2]:
location_to_state = {
    'L1' : 0,
    'L2' : 1,
    'L3' : 2,
    'L4' : 3,
    'L5' : 4,
    'L6' : 5,
    'L7' : 6,
    'L8' : 7,
    'L9' : 8
}

state_to_location = dict((state,location) for location, state in location_to_state.items())

### Create the actions & rewards

In [3]:
actions = [0,1,2,3,4,5,6,7,8]

In [4]:
rewards = np.array([[0,1,0,0,0,0,0,0,0],
                   [1,0,1,0,1,0,0,0,0],
                   [0,1,0,0,0,1,0,0,0],
                   [0,0,0,0,0,0,1,0,0],
                   [0,1,0,0,0,0,0,1,0],
                   [0,0,1,0,0,0,0,0,0],
                   [0,0,0,1,0,0,0,1,0],
                   [0,0,0,0,1,0,1,0,1],
                   [0,0,0,0,0,0,0,1,0]])

### Def remaining hyperparameters

In [11]:
# initialize the parameters
gamma = 0.99 # discount factor
alpha = 0.7  # learning rate
epsilon = 0.1 # Higher value encourages more exploration, lower value encourages more exploitation

### Define agent and its attributes

In [9]:
class QAgent():
    # initialize everything
    def __init__(self, alpha, gamma, epsilon, location_to_state, actions, rewards, state_to_location):
        self.gamma = gamma
        self.alpha = alpha
        self.epsilon = epsilon
        
        self.location_to_state = location_to_state
        self.actions = actions
        self.rewards = rewards
        self.state_to_location = state_to_location
        
        # remember, the Q-value table is of size all actions x all states
        M = len(location_to_state)
        self.Q = np.zeros((M,M), dtype = None, order = 'C')
        
    # now, implement the training method for the agent
    def training(self, start_location, end_location, iterations):

        rewards_new = np.copy(self.rewards)

        ending_state = self.location_to_state[end_location]
        
        rewards_new[ending_state, ending_state] = 999
        
        # DEBUG
        print(rewards_new)

        # pick random current state
        # iterations = the # of training cycles
        for i in range(iterations):
            current_state = np.random.randint(0,9)
            playable_actions = []

            # iterate thru the rewards matrix to get states
            # that are really reachable from the randomly chosen
            # state and assign only those in a list since they are really playable
            for j in range(9):
                if rewards_new[current_state, j] > 0:
                    playable_actions.append(j)

            # choosing next random state using epsilon-greedy strategy
            if np.random.uniform(0, 1) < self.epsilon:
                # Explore: choose a random action from playable_actions
                if len(playable_actions) != 0:
                    next_state = np.random.choice(playable_actions)
                else:
                    # If no playable actions, stay in current state (or handle as an edge case)
                    next_state = current_state
            else:
                # Exploit: choose the action with the highest Q-value
                # If there are no playable actions from current_state, Q[current_state, :] will be all zeros.
                # In such a case, argmax will return 0. We need to ensure we only pick valid actions.
                q_values_for_current_state = self.Q[current_state, :]
                valid_q_values = [q_values_for_current_state[action] for action in playable_actions]

                if valid_q_values:
                    # Get the index of the max Q-value among playable actions
                    max_q_index = np.argmax(valid_q_values)
                    # Map this index back to the actual state in playable_actions
                    next_state = playable_actions[max_q_index]
                else:
                    # Fallback if no valid actions (should ideally not happen with proper environment)
                    next_state = current_state
    
            # finding the difference in Q, often referred to as temporal difference
            # by means of the Bellman's equation (compare with slides)
            TD = rewards_new[current_state, next_state] + self.gamma * self.Q[next_state, np.argmax(self.Q[next_state,])] - self.Q[current_state, next_state]
            self.Q[current_state, next_state] += self.alpha*TD
            # DEBUG
            #print(f"Q[{current_state}, {next_state}]:", self.Q[current_state, next_state])

        route = [start_location]
        next_location = start_location

        # print the optimal route from start to end
        self.get_optimal_route(start_location, end_location, next_location, route, self.Q)

    # compute the optimal route
    def get_optimal_route(self, start_location, end_location, next_location, route, Q):
        while(next_location != end_location):
            starting_state = self.location_to_state[start_location]
            next_state = np.argmax(Q[starting_state,])
            next_location = self.state_to_location[next_state]
            route.append(next_location)
            start_location = next_location
        # DEBUG
        print('Q-table:',Q)
        print("optimal route:", route)

In [10]:
qagent = QAgent(alpha, gamma, epsilon, location_to_state, actions, rewards, state_to_location)
qagent.training('L9', 'L4', 1000)

[[  0   1   0   0   0   0   0   0   0]
 [  1   0   1   0   1   0   0   0   0]
 [  0   1   0   0   0   1   0   0   0]
 [  0   0   0 999   0   0   1   0   0]
 [  0   1   0   0   0   0   0   1   0]
 [  0   0   1   0   0   0   0   0   0]
 [  0   0   0   1   0   0   0   1   0]
 [  0   0   0   0   1   0   1   0   1]
 [  0   0   0   0   0   0   0   1   0]]
Q-table: [[0.00000000e+00 5.26779245e+01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [4.69325569e+01 0.00000000e+00 5.22599109e+01 0.00000000e+00
  2.14877412e+01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 5.27350248e+01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 4.31268007e+01 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 3.57360853e+04
  0.00000000e+00 0.00000000e+00 2.54732734e+01 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 5.27184300e+01 0.00000000e+00 0.00000000e+00
  0.00000

## Hyperparameter Exploration and Epsilon-Greedy Strategy

Below are examples of how you can experiment with the hyperparameters and the newly implemented epsilon-greedy strategy.

### Experimenting with `epsilon`:
A higher `epsilon` value encourages more exploration, which can be beneficial in complex environments to discover new paths, but might slow down convergence. A lower `epsilon` emphasizes exploitation (choosing the best-known action), leading to faster convergence once good paths are found.

In [12]:
# Create a new QAgent instance with a higher epsilon for more exploration
qagent_explore = QAgent(alpha, gamma, 0.5, location_to_state, actions, rewards, state_to_location)
print("\n--- Training with Epsilon = 0.5 (More Exploration) ---")
qagent_explore.training('L9', 'L4', 1000)


--- Training with Epsilon = 0.5 (More Exploration) ---
[[  0   1   0   0   0   0   0   0   0]
 [  1   0   1   0   1   0   0   0   0]
 [  0   1   0   0   0   1   0   0   0]
 [  0   0   0 999   0   0   1   0   0]
 [  0   1   0   0   0   0   0   1   0]
 [  0   0   1   0   0   0   0   0   0]
 [  0   0   0   1   0   0   0   1   0]
 [  0   0   0   0   1   0   1   0   1]
 [  0   0   0   0   0   0   0   1   0]]
Q-table: [[    0.         38325.45766693     0.             0.
      0.             0.             0.             0.
      0.        ]
 [37245.4865412      0.         37644.0535499      0.
  38722.62123183     0.             0.             0.
      0.        ]
 [    0.         38263.12866246     0.             0.
      0.         35423.18387945     0.             0.
      0.        ]
 [    0.             0.             0.         42948.47367375
      0.             0.         39278.36071109     0.
      0.        ]
 [    0.         36736.02601753     0.             0.
      0.         

### Experimenting with `alpha` (learning rate):
`alpha` determines how much new information overrides old information. A higher `alpha` means the agent learns more quickly from recent experiences, while a lower `alpha` makes learning slower but potentially more stable.

In [13]:
# Create a new QAgent instance with a lower alpha for slower, more stable learning
qagent_slow_learn = QAgent(0.1, gamma, epsilon, location_to_state, actions, rewards, state_to_location)
print("\n--- Training with Alpha = 0.1 (Slower Learning) ---")
qagent_slow_learn.training('L9', 'L4', 1000)


--- Training with Alpha = 0.1 (Slower Learning) ---
[[  0   1   0   0   0   0   0   0   0]
 [  1   0   1   0   1   0   0   0   0]
 [  0   1   0   0   0   1   0   0   0]
 [  0   0   0 999   0   0   1   0   0]
 [  0   1   0   0   0   0   0   1   0]
 [  0   0   1   0   0   0   0   0   0]
 [  0   0   0   1   0   0   0   1   0]
 [  0   0   0   0   1   0   1   0   1]
 [  0   0   0   0   0   0   0   1   0]]
Q-table: [[0.00000000e+00 6.00679914e+03 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [2.15999147e-01 0.00000000e+00 5.71452384e+01 0.00000000e+00
  6.87110901e+03 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 6.11665617e+03 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.21480067e+03 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 9.96230520e+03
  0.00000000e+00 0.00000000e+00 1.81971258e+03 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 8

### Experimenting with `gamma` (discount factor):
`gamma` controls the importance of future rewards. A `gamma` closer to 1 makes the agent consider long-term rewards more heavily, suitable for environments where delayed rewards are significant. A `gamma` closer to 0 makes the agent focus on immediate rewards.

In [14]:
# Create a new QAgent instance with a lower gamma for more immediate rewards focus
qagent_immediate_rewards = QAgent(alpha, 0.5, epsilon, location_to_state, actions, rewards, state_to_location)
print("\n--- Training with Gamma = 0.5 (More Immediate Rewards Focus) ---")
qagent_immediate_rewards.training('L9', 'L4', 1000)


--- Training with Gamma = 0.5 (More Immediate Rewards Focus) ---
[[  0   1   0   0   0   0   0   0   0]
 [  1   0   1   0   1   0   0   0   0]
 [  0   1   0   0   0   1   0   0   0]
 [  0   0   0 999   0   0   1   0   0]
 [  0   1   0   0   0   0   0   1   0]
 [  0   0   1   0   0   0   0   0   0]
 [  0   0   0   1   0   0   0   1   0]
 [  0   0   0   0   1   0   1   0   1]
 [  0   0   0   0   0   0   0   1   0]]
Q-table: [[0.00000000e+00 6.43750000e+01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [2.00000000e+00 0.00000000e+00 1.94365451e+00 0.00000000e+00
  1.26750000e+02 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 1.40000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 2.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.99800000e+03
  0.00000000e+00 0.00000000e+00 5.00999855e+02 0.00000000e+00
  0.00000000e+00]
 [0.0

In [19]:
# Example: Make the path from L1 to L2 more rewarding initially
rewards_modified = np.copy(rewards)
rewards_modified[0, 1] = 5 # L1 to L2 now has a higher initial reward
rewards_modified[1, 0] = 20 # And from L2 to L1 now counts as goal

In [18]:
qagent_rewards = QAgent(alpha, gamma, epsilon, location_to_state, actions, rewards_modified, state_to_location)
print("\n--- Training with Modified Rewards (L1-L2 more rewarding) ---")
qagent_rewards.training('L9', 'L4', 1000)


--- Training with Modified Rewards (L1-L2 more rewarding) ---
[[  0   5   0   0   0   0   0   0   0]
 [ 20   0   1   0   1   0   0   0   0]
 [  0   1   0   0   0   1   0   0   0]
 [  0   0   0 999   0   0   1   0   0]
 [  0   1   0   0   0   0   0   1   0]
 [  0   0   1   0   0   0   0   0   0]
 [  0   0   0   1   0   0   0   1   0]
 [  0   0   0   0   1   0   1   0   1]
 [  0   0   0   0   0   0   0   1   0]]
Q-table: [[    0.         45771.59750815     0.             0.
      0.             0.             0.             0.
      0.        ]
 [33926.31462309     0.             0.             0.
  46314.80912919     0.             0.             0.
      0.        ]
 [    0.         45844.75776808     0.             0.
      0.         39454.8441217      0.             0.
      0.        ]
 [    0.             0.             0.         50064.23958807
      0.             0.         42735.56208539     0.
      0.        ]
 [    0.         26649.68399155     0.             0.
      0.  

In [22]:
qagent_rewards = QAgent(0.1, 0.99, 0.99, location_to_state, actions, rewards_modified, state_to_location)
print("\n--- Training with Modified Rewards (L1-L2 more rewarding) ---")
qagent_rewards.training('L9', 'L4', 1000)


--- Training with Modified Rewards (L1-L2 more rewarding) ---
[[  0   5   0   0   0   0   0   0   0]
 [ 20   0   1   0   1   0   0   0   0]
 [  0   1   0   0   0   1   0   0   0]
 [  0   0   0 999   0   0   1   0   0]
 [  0   1   0   0   0   0   0   1   0]
 [  0   0   1   0   0   0   0   0   0]
 [  0   0   0   1   0   0   0   1   0]
 [  0   0   0   0   1   0   1   0   1]
 [  0   0   0   0   0   0   0   1   0]]
Q-table: [[   0.         1415.67853996    0.            0.            0.
     0.            0.            0.            0.        ]
 [ 724.94515868    0.          385.26815086    0.         1702.19305335
     0.            0.            0.            0.        ]
 [   0.         1119.95775277    0.            0.            0.
   513.72578914    0.            0.            0.        ]
 [   0.            0.            0.         5726.37377768    0.
     0.         3691.33198619    0.            0.        ]
 [   0.         1073.90689148    0.            0.            0.
     0.     

### Conclusion from Final Training Run

I was curious about whether the agent will select a much more different route when I give a really high reward for a specific location. 
So I adjusted the hyperparameters in a way that makes exploration important: (`alpha = 0.1`, `gamma = 0.99`, `epsilon = 0.99`) and increased the reward for the transition from `L1` to `L2` (`rewards_modified[0, 1] = 5`) and significantly increased the reward for the transition from `L2` to `L1` (`rewards_modified[1, 0] = 20`).
Specifically the combination of a very high epsilon and the high reward to go from `L1` to `L2` made me hope for altering the optimal route.

But given the structure of the environment, the optimal route from `L9` to `L4` is typically `L9 -> L8 -> L7 -> L4`. This path does not directly involve locations `L1` or `L2`. Therefore, even with a very high reward for the `L2 -> L1` transition, it is *unlikely* that the optimal path to `L4` would change unless `L1` or `L2` were to become an advantageous intermediate step leading to `L4`.

The chosen hyperparameters (`alpha=0.1` for stable learning, `gamma=0.99` for valuing future rewards, and `epsilon=0.99` for a very preferred exploration) contribute to a robust learning process. However, the agent's behavior is fundamentally constrained by the environment's topology as defined by the rewards matrix.

**Conclusion**:
The high reward for `L2 -> L1` primarily influences paths that directly utilize this transition. For a goal like `L4`, which is not directly connected or efficiently reachable via `L1` or `L2` within the current environment, this specific reward modification may not alter the established optimal route. This emphasizes that while hyperparameters and individual rewards are critical, the overall connectivity and reward structure of the environment ultimately dictate the optimal strategies learned by the agent.
